In [1]:
from google.cloud import aiplatform

def export_model_sample(
    project: str,
    model_id: str,
    gcs_destination_output_uri_prefix: str,
    location: str = "us-central1",
    api_endpoint: str = "us-central1-aiplatform.googleapis.com",
    timeout: int = 300,
):
    # The AI Platform services require regional API endpoints.
    client_options = {"api_endpoint": api_endpoint}
    # Initialize client that will be used to create and send requests.
    # This client only needs to be created once, and can be reused for multiple requests.
    client = aiplatform.gapic.ModelServiceClient(client_options=client_options)
    output_config = {
        "artifact_destination": {
            "output_uri_prefix": gcs_destination_output_uri_prefix
        },
        # For information about export formats: https://cloud.google.com/ai-platform-unified/docs/export/export-edge-model#aiplatform_export_model_sample-drest
        "export_format_id": "tf-saved-model",
    }
    name = client.model_path(project=project, location=location, model=model_id)
    response = client.export_model(name=name, output_config=output_config)
    print("Long running operation:", response.operation.name)
    print("output_info:", response.metadata.output_info)
    export_model_response = response.result(timeout=timeout)
    print("export_model_response:", export_model_response)


ImportError: cannot import name 'aiplatform' from 'google.cloud' (unknown location)

In [ ]:
gcloud ai endpoints export \
  --region=europe-north1 \
  --model=transformations-all-features-scaled-target.json\
  --output-uri=gs://cloud-ai-platform-63a5e957-921e-4e42-9b23-8c38a4a83e6a/transformations-all-features-scaled-target.json

In [3]:
gcloud ai endpoints export --region=<REGION> --model=<MODEL_NAME> --output-uri=https://storage.cloud.google.com/cloud-ai-platform-63a5e957-921e-4e42-9b23-8c38a4a83e6a/transformations-all-features-scaled-target.json


SyntaxError: invalid syntax (1988870599.py, line 1)

In [3]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras

In [4]:
import tensorflow as tf
print(tf.__version__)

2.16.1


In [13]:
from tensorflow.python.framework import load_library
#load_library.load_op_library('/path/to/tf_decode_proto_sparse_v4.so')

In [15]:
model = tf.keras.models.load_model('/Users/clara/Desktop/neuefische/d-drivers/final_model_vertex/tf-saved-model/2024-04-21T13:24:51.408713Z/predict/001/')

ValueError: File format not supported: filepath=/Users/clara/Desktop/neuefische/d-drivers/final_model_vertex/tf-saved-model/2024-04-21T13:24:51.408713Z/predict/001/. Keras 3 only supports V3 `.keras` files and legacy H5 format files (`.h5` extension). Note that the legacy SavedModel format is not supported by `load_model()` in Keras 3. In order to reload a TensorFlow SavedModel as an inference-only layer in Keras 3, use `keras.layers.TFSMLayer(/Users/clara/Desktop/neuefische/d-drivers/final_model_vertex/tf-saved-model/2024-04-21T13:24:51.408713Z/predict/001/, call_endpoint='serving_default')` (note that your `call_endpoint` might have a different name).

In [14]:
import os
import tensorflow as tf

# Define the path to the saved model
model_path = '/Users/clara/Desktop/neuefische/d-drivers/final_model_vertex/tf-saved-model/2024-04-21T13:24:51.408713Z/predict/001/'

# Check if the directory exists
if os.path.exists(model_path):
    # List the files in the directory
    files = os.listdir(model_path)
    
    # Print the files to verify if 'saved_model.pb' is present
    print(files)
    
    # Attempt to load the saved model
    try:
        model = tf.saved_model.load(model_path)
        print("Model loaded successfully!")
    except Exception as e:
        print(f"Error loading the model: {e}")
else:
    print(f"The directory {model_path} does not exist.")


['assets.extra', 'variables', 'saved_model.pb', 'assets']
Error loading the model: Op type not registered 'DecodeProtoSparseV4' in binary running on Air-von-Clara.fritz.box. Make sure the Op and Kernel are registered in the binary running in this process. Note that if you are loading a saved graph which used ops from tf.contrib (e.g. `tf.contrib.resampler`), accessing should be done before importing the graph, as contrib ops are lazily registered when the module is first accessed.


In [ ]:
# Create a new TensorFlow graph
graph = tf.Graph()

with graph.as_default():
    # Load the model
    loaded_model = tf.saved_model.load('/Users/clara/Desktop/neuefische/d-drivers/tf-saved-model/2024-04-12T07:29:42.410616Z/lower_bound/001')

# Use the loaded model for inference
# For example:
# predictions = loaded_model(input_data)


RuntimeError: Op type not registered 'DecodeProtoSparseV4' in binary running on Air-von-Clara. Make sure the Op and Kernel are registered in the binary running in this process. Note that if you are loading a saved graph which used ops from tf.contrib (e.g. `tf.contrib.resampler`), accessing should be done before importing the graph, as contrib ops are lazily registered when the module is first accessed.

In [13]:
# Create a new TensorFlow graph
graph = tf.Graph()

with graph.as_default():
    # Load the model
    loaded_model = tf.saved_model.load('/Users/clara/Desktop/neuefische/d-drivers/tf-saved-model/2024-04-12T07:29:42.410616Z/lower_bound/001')

# Use the loaded model for inference
# For example:
# predictions = loaded_model(input_data)


RuntimeError: Op type not registered 'DecodeProtoSparseV4' in binary running on Air-von-Clara. Make sure the Op and Kernel are registered in the binary running in this process. Note that if you are loading a saved graph which used ops from tf.contrib (e.g. `tf.contrib.resampler`), accessing should be done before importing the graph, as contrib ops are lazily registered when the module is first accessed.

In [ ]:
# Make predictions
predictions = loaded_model(inputs)

In [ ]:
import streamlit as st
from typing import Dict
from google.cloud import aiplatform
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
from sklearn.preprocessing import PowerTransformer
import pickle

# Initialize PowerTransformer
#pt = PowerTransformer()
file_path_specific = '/Users/clara/Desktop/neuefische/d-drivers/notebooks/power_transformer_ext_impr.pkl'
with open(file_path_specific, 'rb') as file:
    loaded_pt = pickle.load(file)

# Function to reverse power transformation
def reverse_power_transformation(predicted_value, pt):
    # Reshape the predicted value for inverse transformation
    predicted_value_transformed = predicted_value.reshape(-1, 1)
    
    # Inverse transform the predicted value
    return loaded_pt.inverse_transform(predicted_value_transformed)

# Streamlit app
st.title("Vertex AI Prediction Demo")

# Text areas for Title, Abstract, and Article
title_text = st.text_area("Title", "Your Title Here")
abstract_text = st.text_area("Abstract", "Your Abstract Here")
article_text = st.text_area("Article", "Your Article Here")

# Input features
classification_product = st.selectbox("Classification Product", ['E-Auto', 'Auto', 'Zubehör', 'Motorrad', 'Energie', 'Verkehr','Wallbox/Laden', 'Solaranlagen', 'E-Bike', 'Fahrrad', 'E-Scooter','Solarspeicher', 'Balkonkraftwerk', 'Solargenerator', 'THG','Wärmepumpe', 'Versicherung'])
classification_type = st.selectbox("Classification Type", ['Ratgeber', 'News', 'Kaufberatung', 'Deal', 'Test','Erfahrungsbericht', 'Video'])
urls_per_days = st.select_slider("Publishing Frequency",options=['0','0.25','0.5','0.75','1','1.25','1.5','1.75', '2'])
#video_standard_and_widget = st.selectbox("Video Standard and Widget", ["True", "False"])
media_type = st.selectbox("Media Type", ["Image","Video"])
if media_type == 'Video':
    video_widget = st.selectbox("Self-produced video", ["True", "False"])
else:
    video_widget = 'False'
YOUR_N_DAYS_VALUE = st.number_input("Expected time the article is online", 100)

# Count characters in Title, Abstract, and Article
h1_len = len(title_text)
meta_title_len = len(title_text)
abstract_len = len(abstract_text) 
meta_desc_len = abstract_len - 100
word_count = abstract_len+ len(article_text)

# Prepare instance dictionary
instance_dict = {
    "word_count": str(word_count),
    "classification_product": classification_product,
    "classification_type": classification_type,
    "urls_per_days": urls_per_days,
    "meta_title_len": str(meta_title_len),
    "meta_desc_len": str(meta_desc_len),
    "h1_len": str(h1_len),
    "abstract_len": str(abstract_len),
    "google_trend_label": str('elektroauto'),
    "google_trend_score": str(31.0),
    "video_standard_and_widget": str('False') if video_widget == 'True' else str('True'),
    "video_widget": str('True') if video_widget == 'True' else str('False'),
    "media_type_other": str('False'),
    "media_type_video": str('True') if media_type == 'Video' else str('False'),
    "Authors": str("lemur")
}


def predict_tabular_regression_sample(
    project: str,
    endpoint_id: str,
    instance_dict: Dict,
    location: str = "us-central1",
    api_endpoint: str = "us-central1-aiplatform.googleapis.com",
):
    # The AI Platform services require regional API endpoints.
    client_options = {"api_endpoint": api_endpoint}
    
    # Initialize client that will be used to create and send requests.
    client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)
    
    # Parse the instance dictionary
    instance = json_format.ParseDict(instance_dict, Value())
    instances = [instance]
    
    # Prepare empty parameters
    parameters_dict = {}
    parameters = json_format.ParseDict(parameters_dict, Value())
    
    # Construct the endpoint path
    endpoint = client.endpoint_path(
        project=project, location=location, endpoint=endpoint_id
    )
    
    try:
        # Send the prediction request
        response = client.predict(
            endpoint=endpoint, instances=instances, parameters=parameters
        )

        # Display the response
        st.write("Response:")
        st.write(f"Deployed Model ID: {response.deployed_model_id}")
        
        # Print the predictions
        predictions = response.predictions
        for prediction in predictions:
            st.write("Prediction:", dict(prediction))
            
    except Exception as e:
        st.write(f"Error during prediction: {e}")
        st.write(f"Error type: {type(e)}")


# Button to trigger prediction
if st.button("Predict"):
    response = predict_tabular_regression_sample(
        project="101568381799",
        endpoint_id="5222247024354656256",
        instance_dict=instance_dict,
        location="us-central1",
        api_endpoint="us-central1-aiplatform.googleapis.com"
        )

    # Extract prediction value from the response
    prediction_value = response.predictions[0]  # Assuming you're interested in the first prediction
    st.write(f"Predicted Value: {prediction_value}")

    # Reverse Power Transformation
    reversed_value_transformed = reverse_power_transformation(predicted_value, loaded_pt)
    
    # Reverse Normalization by multiplying through n_days
    n_days = YOUR_N_DAYS_VALUE 
    reversed_value = reversed_value_transformed * n_days
    
    # Display the reversed value
    st.write(f"Predicted Value (Original Scale): {reversed_value}")




In [ ]:
import pandas as pd
import numpy as np
from pycaret.regression import *

# Assuming you have set up your data and trained the model as shown in your code
# num_bl and cat_bl contain the numeric and categorical feature names, respectively
# target contains the target variable name

# Define the input vector
inp_vector = {
    'word_count': [1000],
    'urls_per_days': [0.25],
    'meta_title_len': [60],
    'meta_desc_len': [150],
    'h1_len': [40],
    'abstract_len': [200],
    'google_trend_score': [31.0],
    'Authors': ['lemur'],
    'media_type_video': ['True'],
    'video_widget': ['True'],
    'video_standard_and_widget': ['False'],
    'google_trend_label': ['elektroauto'],
    'classification_type': ['News'],
    'classification_product': ['E-Auto']
}

# Convert the dictionary to a DataFrame
inp_df = pd.DataFrame.from_dict(inp_vector)

# One-hot encode categorical variables
inp_df = pd.get_dummies(inp_df, columns=cat_bl, drop_first=True)

# Ensure all columns in cat_bl are present in inp_df after one-hot encoding
missing_columns = set(cat_bl) - set(inp_df.columns)
for col in missing_columns:
    inp_df[col] = 0

# Define the range of publishing frequencies
publ_freq_range = np.linspace(0.001, 0.45, 100)

# Initialize an empty list to store predictions
predictions = []

# Loop through the range of publishing frequencies and make predictions
for pf in publ_freq_range:
    inp_df['urls_per_days'] = pf
    pred = predict_model(s, data=inp_df)
    predictions.append(pred['Label'][0])  # Assuming the predicted value is stored in the 'Label' column

# Now, predictions contains the predicted values for the range of publishing frequencies


AttributeError: 'RegressionExperiment' object has no attribute 'predict'